In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
from io import BytesIO
from importlib import reload
from solutions.lib import run

# Refactor

lib.py was refactored ... 

In [6]:
ADDRESS = ('217.19.216.210', 8333)

In [7]:
from solutions.network import *

peer = PeerConnection(*ADDRESS)

peer.handshake()

Our goal is initial block download on the first 10,000 bitcoin blocks. Bitcoin does initial block download by first downloading "block headers" -- which are blocks without the transactions -- and validating their hashes form and chain and that they satisfy bitcoin's proof of work requirements. Once the longest valid chain of block headers has been discovered and validated, then the bitcoin software proceeds to download and the full blocks -- block headers plus all the transactions. This takes a lot longer, but can be downloaded in parallel from multiple peers, which speeds up the process. 

First of all, we need to figure out how to download block headers. The bitcoin wiki tells us how: behold the We need to send a getheaders message to request the [`getheaders`](https://en.bitcoin.it/wiki/Protocol_documentation#getheaders) network message.

# `getheaders`

[network.py](./network.py) contains a stub for a `GetHeadersMessage` class. Let's fill it out so that we can send a `getheaders` message in the cell below and get a first batch of headers from a bitcoin peer.

In [8]:
import solutions.network as network

reload(network)
run(network.GetHeadersMessageTest("test_serialize"))

.
----------------------------------------------------------------------
Ran 1 test in 0.005s

OK


Now that we have a class abstracting the `getheaders` message, let's put it to use. Use this message to get the next block after genesis ...

In [42]:
from solutions.network import HeadersMessageInitial as HeadersMessage

# connect to peer
peer = PeerConnection(*ADDRESS, logging=True)
peer.handshake()

# construct the message
genesis_hash_hex = '000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f'
genesis_hash_bytes = bytes.fromhex(genesis_hash_hex)
getheaders = GetHeadersMessage(start_block=genesis_hash_bytes)

# send the message
peer.send(getheaders)

# see what we get back
while True:
    peer.read()

sending: version: 7f1101000000000000000000bb8edb5c00000000000000000000000000000000000000000000ffff00000000208d000000000000000000000000000000000000ffff00000000208d706d64938f087e06182f70726f6772616d6d696e67626974636f696e3a302e312f0000000000
receiving: version: 7f1101000d04000000000000bc8edb5c00000000000000000000000000000000000000000000ffff4a493663a7840d04000000000000000000000000000000000000000000000000181f4628d8a1825f102f5361746f7368693a302e31372e312f66ca080001
sending: verack: 
receiving: verack: 
sending: getheaders: 7f110100016fe28c0ab6f1b372c1a6a246ae63f74f931e8365e15a089c68d61900000000000000000000000000000000000000000000000000000000000000000000000000
receiving: sendheaders: 
receiving: sendcmpct: 000200000000000000
receiving: sendcmpct: 000100000000000000
receiving: ping: 15bdb20aa0d3b1c3
receiving: feefilter: e803000000000000
receiving: headers: ...


KeyboardInterrupt: 

In [40]:
import solutions.network as network

reload(network)
run(network.HeadersMessageTest("test_parse"))

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


In [ ]:
from solutions.network import HeadersMessageInitial as HeadersMessage

# construct the message
genesis_hash_hex = '000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f'
genesis_hash_bytes = bytes.fromhex(genesis_hash_hex)
getheaders = GetHeadersMessage(start_block=genesis_hash_bytes)

# send the message
peer.send(getheaders)

# wait for the "headers" response
headers_payload = peer.wait_for(HeadersMessage)
headers_payload

Now that we need to learn to interpret the `headers` response we are getting. Let's implement `HeadersMessage.parse` to make this happen. If we look at the protocol docs we see that the payload of the `headers` message contains, as we should expect, block header objects. In order to implement `HeadersMessage.parse` we first need to implement implement a `BlockHeader` class with a `.parse` method which `HeadersMessage.parse` can call.

### `BlockHeader`

block.py has the stub of a `BlockHeader` class. Fill out the `BlockHeader.parse` method to get the test below to pass.

In [10]:
import solutions.network as network

reload(network)
run(network.HeadersMessageTest("test_parse"))

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


Now that we have a `HeadersMessage` that passes the tests, let's use it to interpret the `headers` payload bytes we received earlier:

In [11]:
from solutions.network import HeadersMessage

headers_msg = HeadersMessage.parse(BytesIO(headers_payload))
headers_msg.headers

 ...]

In [12]:
headers_msg.headers[0]

In [13]:
headers_msg.headers[0].prev_block

b'\x00\x00\x00\x00\x00\x19\xd6h\x9c\x08Z\xe1e\x83\x1e\x93O\xf7c\xaeF\xa2\xa6\xc1r\xb3\xf1\xb6\n\x8c\xe2o'

Very good! Now let's try to call this repeatedly and build up a chain of block headers:

In [71]:
from solutions.block import RAW_GENESIS_BLOCK

# Create a list of headers with genesis block filled in
GENESIS_BLOCK = BlockHeader.parse(BytesIO(RAW_GENESIS_BLOCK))
headers = [GENESIS_BLOCK]

# connect to our peer
peer = PeerConnection(*ADDRESS)
peer.handshake()

while len(headers) < 10000:
    # construct the message
    getheaders = GetHeadersMessage(start_block=headers[-1].hash())

    # send the message
    peer.send(getheaders)

    # wait for the "headers" response
    headers_msg = peer.wait_for(HeadersMessage)
    print(f'received {len(headers_msg.headers)} headers')
    
    # append block headers received to headers array
    for header in headers_msg.headers:
        headers.append(header)

AttributeError: 'bytes' object has no attribute 'headers'

With a bit long list of block headers, we can now validate that they actually form a chain.

The block id's that we are familiar from using block explorers aren't actually an attribute of the blocks themselves. 

![img](./genesis.png)

![img](./block.png)

Rather, they are derived from all the other fields listed above by SHA256 hashing the block twice, and interpreting this as a little-endian hexidecimal number.

So we need to implement this in order to derive `BlockHeader` id's.

For this we will need to implement three methods:
- `BlockHeader.serialize` -- which turns a `BlockHeader` into `bytes`
- `BlockHeader.hash` -- which SHA256 hashes result of `BlockHeader.serialize()` twice
- `BlockHeader.id`: which interprets `BlockHeader.hash` as hexidecimal

First, implement `BlockHeader.serialize` to get this test passing:

In [15]:
import solutions.block as block

reload(block)
run(block.BlockHeaderTest("test_serialize"))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


Next, implement `BlockHeader.hash` to get this test passing:

In [16]:
import solutions.block as block

reload(block)
run(block.BlockHeaderTest("test_hash"))

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


Finally, implement `BlockHeader.id` to get this test passing:

In [17]:
import solutions.block as block

reload(block)
run(block.BlockHeaderTest("test_id"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


Now magically the headers we downloaded earlier have id's:

In [18]:
for header in headers[:10]:
    print(header.id())

000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f
00000000839a8e6886ab5951d76f411475428afc90947ee320161bbf18eb6048
000000006a625f06636b8bb6ac7b960a8d03705d1ace08b1a19da3fdcc99ddbd
0000000082b5015589a3fdf2d4baff403e6f0be035a5d9742c1cae6295464449
000000004ebadb55ee9096c9a2f8880e09da59c0d68b1c228da88e48844a1485
000000009b7262315dbf071787ad3656097b892abffd1f95a1a022f896f533fc
000000003031a0e73735690c5a1ff2a4be82553b2a12b776fbd3a215dc8f778d
0000000071966c2b1d065fd446b1e485b2c9d9594acd2007ccbd5441cfc89444
00000000408c48f847aa786c2268fc3e6ec2af68e8468a34a28c61b7f1de0dc6
000000008d9dc510f23c2657fc4f67bea30078cc05a90eb89e84cc475c080805


Let's verify that they form a chain. Since `BlockHeader.prev_block` is bytes let's use `BlockHeader.hash()` to compare these directly

In [19]:
for height, header in enumerate(headers[1:], 1):
    prev_header = headers[height-1]
    assert header.prev_block == prev_header.hash()

print('the chain is good!')

the chain is good!


### Proof-of-Work

Now that we can validate the headers form a chain, let's validate the chain they form is ***valid***.

A few things are involved here.

First, we need to calculate the current difficulty. We take the `BlockHeader.bits` field and apply this formula: `coefficient * 256**(exponent - 3)`, where coefficient is the little-endian interpretation of the first 3 bytes of `BlockHeader.bits` and `exponent` is the last byte of `BlockHeader.bits`

Fill out the `bits_to_target` function in `lib.py` to do this. Get the following test to pass:

In [20]:
import solutions.lib as lib

reload(lib)
run(lib.LibraryTest("test_bits_to_target"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


Now that we can calculate the target for blocks headers, see if the `nonce` provided by the blocks leads to a hash that, when interpreted as a number, is less than the target.

Update `BlockHeader.check_pow()` to get the following test to pass:

In [21]:
import solutions.block as block

reload(block)
run(block.BlockHeaderTest("test_check_pow"))

.
----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


Now let's update our code that checked whether a chain was formed to also check that proof-of-work was satisifed for each block header.

In [22]:
for height, header in enumerate(headers[1:], 1):
    # check that a headers form a chain
    prev_header = headers[height-1]
    assert header.prev_block == prev_header.hash()

    # check that headers satisfy proof-of-work
    assert header.check_pow()

print('the chain is good!')

the chain is good!


(also assert that `BlockHeader.bits` is always equal to the correct initial bits or something along these lines ...)

# Fetching Blocks

We now have a chain of block headers. It's time to get the full blocks -- namely, the blocks headers *plus the transactions*.

To do this, we need to send a [`getdata`](https://en.bitcoin.it/wiki/Protocol_documentation#getdata) message. This message is how you look up stuff on the bitcoin p2p network. To use it, you must construct an ["inventory vector"](https://en.bitcoin.it/wiki/Protocol_documentation#Inventory_Vectors) containing entries for each of the objects you'd like to look up. An inventory vector is simple: each item contains the id of the whatever data you're requesting (in our case block hashes) and a marker indicating what type of data this is (e.g. block or transaction).



In [23]:
import solutions.network as network

reload(network)
run(network.GetDataMessageTest("test_serialize"))

.
----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


Let's see what we get back when we send one of these:

In [38]:
from solutions.network import GetDataMessage

# connect to peer
peer = PeerConnection(*ADDRESS, logging=True)
peer.handshake()

# send getdata
getdata = GetDataMessage()
getdata.add_block(headers[1].hash())
peer.send(getdata)

# see what we get back
while True:
    peer.read()

sending: version: 7f1101000000000000000000068edb5c00000000000000000000000000000000000000000000ffff00000000208d000000000000000000000000000000000000ffff00000000208da37c459bc2294185182f70726f6772616d6d696e67626974636f696e3a302e312f0000000000
receiving: version: 7f1101000d04000000000000068edb5c00000000000000000000000000000000000000000000ffff4a493663a1d80d040000000000000000000000000000000000000000000000009a0a8f0276c026bd102f5361746f7368693a302e31372e312f66ca080001
sending: verack: 
receiving: verack: 
sending: getdata: 01020000004860eb18bf1b1620e37e9490fc8a427514416fd75159ab86688e9a8300000000
receiving: sendheaders: 
receiving: sendcmpct: 000200000000000000
receiving: sendcmpct: 000100000000000000
receiving: ping: cca4d8eaf891a31b
receiving: feefilter: e803000000000000
receiving: block: 010000006fe28c0ab6f1b372c1a6a246ae63f74f931e8365e15a089c68d6190000000000982051fd1e4ba744bbbe680e1fee14677ba1a3c3540bf7b1cdb606e857233e0e61bc6649ffff001d01e3629901010000000100000000000000000000000000000000000

KeyboardInterrupt: 

# Parsing Blocks

Next, we need to be able to parse full blocks.

This is involved. Blocks contain transactions, so we will need to parse transactions to pull this off.

Since I think you are beginning to get the idea we won't write all the code but we'll write some parsing code for each thing ...

In [31]:
import solutions.tx as tx

reload(tx)
run(tx.TxTest("test_parse_version"))

.
----------------------------------------------------------------------
Ran 1 test in 0.005s

OK


In [36]:
import solutions.script as script

reload(script)
run(script.ScriptTest("test_parse"))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [33]:
import solutions.tx as tx

reload(tx)
run(tx.TxTest("test_parse_inputs"))

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


In [34]:
import solutions.tx as tx

reload(tx)
run(tx.TxTest("test_parse_outputs"))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [35]:
import solutions.tx as tx

reload(tx)
run(tx.TxTest("test_parse_locktime"))

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


Now we can parse blocks. Implement `Block.parse` and get this test to pass:

In [43]:
import solutions.block as block

reload(block)
run(block.BlockTest("test_parse"))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [48]:
import solutions.network as network

reload(network)
run(network.BlockMessageTest("test_parse"))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [53]:
from solutions.network import BlockMessage

# connect to peer
peer = PeerConnection(*ADDRESS)
peer.handshake()

# send getdata
getdata = GetDataMessage()
getdata.add_block(headers[1].hash())
peer.send(getdata)

# wait for the block message
block_msg = peer.wait_for(BlockMessage)
block_msg

In [54]:
block = block_msg.block
block

In [56]:
block.txns

In [58]:
block.txns[0].tx_ins

[0000000000000000000000000000000000000000000000000000000000000000:4294967295]

In [60]:
# first bitcoin ever mined ...
block.txns[0].tx_outs[0]

5000000000:0496b538e853519c726a2c91e61ec11600ae1390813a627c66fb8be7947be63c52da7589379515d4e0a604f8141781e62294721166bf621e73a82cbf2342c858ee OP_CHECKSIG

Now let's download the blocks corresponding to the block headers we downloaded earlier

In [68]:
from solutions.block import Block, RAW_GENESIS_BLOCK

# Create a list of headers with genesis block filled in
    GENESIS_BLOCK = Block.parse(BytesIO(RAW_GENESIS_BLOCK))
blocks = [GENESIS_BLOCK]

# connect to peer
peer = PeerConnection(*ADDRESS)
peer.handshake()

header_index = 1

header_slice = headers[:500]

while header_index <= len(header_slice):
    # prepare and send getdata requesting 100 blocks
    getdata = GetDataMessage()
    chunk = headers[header_index:header_index+100]
    for header in chunk:
        getdata.add_block(header.hash())
    peer.send(getdata)
    
    # wait for 100 blocks to arrive, add to our list of blocks
    for _ in chunk:
        block_msg = peer.wait_for(BlockMessage)
        blocks.append(block_msg.block)
        
    header_index += len(chunk)

    print(len(blocks))

101
201
301
401
501


In [69]:
# the headers match, which guarantees all other 
# BlockHeaders attributes match too

for i in range(len(header_slice)):
    assert headers[i].hash() == blocks[i].hash()

Let's make a `Blockchain` class to abstract everything we've learned so far 